# Getting Elevation Info From the Austrian Elevation Service

[![Binder](http://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/anitagraser/ogd-at-lab/main?urlpath=lab/tree/notebooks/elevation.ipynb)

Homepage of the service: https://maegger.github.io/getAustrianElevation.html (Copyright (c) 2017, Manfred Egger)

Related QGIS plugin: https://github.com/maegger/AustrianElevation

Elevation data source: CC BY 3.0 AT http://geoland.at/ 


In [ ]:
import hvplot.pandas
from geopy.geocoders import Nominatim
from utils.ogc_io import gdf_from_wfs
from utils.plotting import hvplot_with_buffer
from utils.converting import location_to_gdf

In [ ]:
address = "Stephansdom, Wien"
locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode(address)
print(location.address)
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))
gdf = location_to_gdf(location, address)

Before we can query the elevation, we need to reproject the coordinates to EPSG:3857

In [ ]:
gdf = gdf.to_crs('epsg:3857')
gdf

In [ ]:
from os.path import exists
from urllib.request import urlretrieve

def get_elevation(point):
    """
    Retrieve elevation info from the Austrian Elevation Service
    
    Implementation based on https://github.com/maegger/AustrianElevation/blob/6e0f468b6094caace6cd35f00704e4087e851cec/tree/AustrianElevation/AustrianElevation.py#L97
    
    Parameters
    ----------
    point : Shapely Point
        Point in EPSG:3857 
    """
    x = point.x
    y = point.y
    mod_x_path = x % 20000;
    path_x = x - mod_x_path;
    database = int(path_x );
    mod_y = y % 10;
    raster_y = y - mod_y;
    mod_x = x % 10;
    raster_x = int(x - mod_x);
    file = f'{int(raster_y)}.txt'
    url = f"https://raw.githubusercontent.com/maegger/{database}/master/{int(raster_y)}.txt"
    if not exists(file):
        urlretrieve(url, file)
    data = open(file, 'r')
    for line in data:
        x_wert =  int(line.split(' ', 1 )[0])
        if x_wert == raster_x:
            elevationall = line.split(' ', 1 )[1]
            return int(elevationall)

In [ ]:
gdf.loc[0, 'elevation'] = get_elevation(gdf.iloc[0].geometry)
gdf